In [0]:
# ======================================================================================
# BIBLIOTHÈQUES
# ======================================================================================
import logging
from pyspark.sql import DataFrame, SparkSession
from pyspark.sql.functions import col, to_date, date_format

# ======================================================================================
# 1. DÉCLARATION DES PARAMÈTRES (WIDGETS)
# ======================================================================================
dbutils.widgets.text("storage_account", "stsalesinsightcuxm0611", "Nom du compte de stockage")
dbutils.widgets.text("container", "data", "Nom du conteneur")
dbutils.widgets.text("silver_folder", "silver/sales_orders/", "Dossier source dans la couche Silver")

# --- Utilisation d'un scope de secrets unique pour tout le projet ---
dbutils.widgets.text("secret_scope", "dbricks-scope-projet", "Scope unique pour les secrets du projet")

# Paramètres pour les clés des secrets
dbutils.widgets.text("adls_secret_key", "adls-access-key", "Clé du secret pour l'accès ADLS")
dbutils.widgets.text("sql_user_key", "sql-admin-user", "Clé du secret pour l'utilisateur SQL")
dbutils.widgets.text("sql_password_key", "sql-admin-password", "Clé du secret pour le mot de passe SQL")

# Paramètres pour la connexion à Azure SQL
dbutils.widgets.text("jdbc_hostname", "sqlsvr-salesinsightcuxm0611.database.windows.net", "Serveur Azure SQL DB")
dbutils.widgets.text("jdbc_database", "sqldb-salesinsight-gold", "Base de données Gold")


# ======================================================================================
# 2. DÉFINITION DES FONCTIONS
# ======================================================================================

def setup_adls_access(storage_account: str, scope: str, key_name: str) -> None:
    """
    Configure l'accès au compte de stockage ADLS Gen2 pour la session Spark en cours.
    """
    logging.info(f"Configuration de l'accès pour le compte de stockage : {storage_account}")
    access_key = dbutils.secrets.get(scope=scope, key=key_name)
    spark.conf.set(
        f"fs.azure.account.key.{storage_account}.dfs.core.windows.net",
        access_key
    )
    logging.info("Accès ADLS configuré avec succès pour cette session.")

def get_jdbc_connection_properties(hostname: str, db_name: str, scope: str, user_key: str, pwd_key: str) -> tuple[str, dict]:
    """
    Construit l'URL JDBC et le dictionnaire de propriétés pour la connexion à Azure SQL DB.
    """
    logging.info("Configuration de la connexion JDBC.")
    jdbc_port = 1433
    sql_user = dbutils.secrets.get(scope=scope, key=user_key)
    sql_password = dbutils.secrets.get(scope=scope, key=pwd_key)
    jdbc_url = f"jdbc:sqlserver://{hostname}:{jdbc_port};database={db_name}"
    properties = {
      "user": sql_user,
      "password": sql_password,
      "driver": "com.microsoft.sqlserver.jdbc.SQLServerDriver"
    }
    return jdbc_url, properties

# --- CORRECTION: On supprime la fonction 'read_data' trop générique qui causait l'erreur ---

def create_fact_sales(silver_df: DataFrame, dim_product_df: DataFrame, dim_customer_df: DataFrame, dim_date_df: DataFrame) -> DataFrame:
    """
    Crée la table de faits des ventes en joignant les données Silver avec les dimensions.
    """
    logging.info("Début de la création de la table de faits 'FactSales'.")
    
    # a. Préparer le DataFrame Silver pour la jointure
    df_sales = silver_df.withColumn("DateKey", date_format(to_date(col("ORDERDATE")), "yyyyMMdd").cast("int"))

    # b. Joindre avec les dimensions pour obtenir les clés de substitution
    df_joined_prod = df_sales.join(dim_product_df, on="PRODUCTCODE", how="left")
    df_joined_cust = df_joined_prod.join(dim_customer_df, on="CUSTOMERNAME", how="left")
    df_joined_date = df_joined_cust.join(dim_date_df, on="DateKey", how="left")

    # c. Sélectionner les colonnes finales pour la table de faits
    fact_sales_df = df_joined_date.select(
        col("ORDERNUMBER").alias("OrderNumber"),
        col("ProductKey"),
        col("CustomerKey"),
        col("DateKey"),
        col("QUANTITYORDERED").alias("QuantityOrdered"),
        col("PRICEEACH").alias("PriceEach"),
        col("SALES").alias("SalesAmount")
    )
    
    logging.info("Création de 'FactSales' terminée.")
    return fact_sales_df

def write_fact_to_gold(fact_df: DataFrame, table_name: str, jdbc_url: str, properties: dict) -> None:
    """
    Écrit la table de faits dans la couche Gold (Azure SQL DB).
    """
    logging.info(f"Début de l'écriture de la table de faits '{table_name}' vers la couche Gold.")
    try:
        fact_df.write.jdbc(
            url=jdbc_url,
            table=table_name,
            mode="overwrite",
            properties=properties
        )
        logging.info(f"✅ La table de faits '{table_name}' a été écrite avec succès.")
    except Exception as e:
        logging.error(f"❌ ERREUR lors de l'écriture de la table de faits '{table_name}'.", exc_info=True)
        raise e

# ======================================================================================
# 3. POINT D'ENTRÉE PRINCIPAL (MAIN)
# ======================================================================================
if __name__ == "__main__":
    
    logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
    
    logging.info("===================================================")
    logging.info("DÉMARRAGE DU PIPELINE SILVER-TO-GOLD (FactSales)")
    logging.info("===================================================")
    
    try:
        # Récupération des paramètres
        storage_account = dbutils.widgets.get("storage_account").strip()
        container = dbutils.widgets.get("container").strip()
        silver_folder = dbutils.widgets.get("silver_folder").strip()
        secret_scope = dbutils.widgets.get("secret_scope").strip()
        adls_secret_key = dbutils.widgets.get("adls_secret_key").strip()
        jdbc_hostname = dbutils.widgets.get("jdbc_hostname").strip()
        jdbc_database = dbutils.widgets.get("jdbc_database").strip()
        sql_user_key = dbutils.widgets.get("sql_user_key").strip()
        sql_password_key = dbutils.widgets.get("sql_password_key").strip()

        # --- ORCHESTRATION ---
        
        # 1. Configurer les accès
        setup_adls_access(storage_account, secret_scope, adls_secret_key)
        jdbc_url, connection_props = get_jdbc_connection_properties(
            jdbc_hostname, jdbc_database, secret_scope, sql_user_key, sql_password_key
        )
        
        # 2. Lire toutes les sources de données nécessaires
        silver_path = f"abfss://{container}@{storage_account}.dfs.core.windows.net/{silver_folder}"
        logging.info(f"Lecture des données Silver depuis '{silver_path}'...")
        df_silver = spark.read.format("delta").load(silver_path)
        logging.info("Lecture des données Silver réussie.")
        
        # --- CORRECTION: Utilisation explicite de spark.read.jdbc pour chaque table de dimension ---
        logging.info("Lecture des tables de dimension depuis Azure SQL...")
        df_dim_product = spark.read.jdbc(url=jdbc_url, table="DimProduct", properties=connection_props)
        df_dim_customer = spark.read.jdbc(url=jdbc_url, table="DimCustomer", properties=connection_props)
        df_dim_date = spark.read.jdbc(url=jdbc_url, table="DimDate", properties=connection_props)
        logging.info("Lecture des dimensions réussie.")

        # 3. Exécuter la transformation pour créer la table de faits
        fact_sales_dataframe = create_fact_sales(df_silver, df_dim_product, df_dim_customer, df_dim_date)
        
        # 4. Écrire la table de faits dans la base de données Gold
        write_fact_to_gold(fact_sales_dataframe, "FactSales", jdbc_url, connection_props)
        
        logging.info("===================================================")
        logging.info("PIPELINE SILVER-TO-GOLD (FactSales) TERMINÉ AVEC SUCCÈS")
        logging.info("===================================================")

    except Exception as e:
        logging.error("Le pipeline a échoué dans le bloc principal.", exc_info=True)
        raise e
